In [2]:
import pymysql
import pandas as pd
import json
from datetime import datetime, timedelta,date


# TODO implement
# from sshtunnel import SSHTunnelForwarder
conn = pymysql.connect(host='db-chatbot.-.ap-northeast-2.rds.amazonaws.com', 
                     port = 3306,
                     user='user',
                     database='DB',
                     password = PS',
                     charset='utf8')  


cursor = conn.cursor()

query = '''SELECT el.createdAt,el.botId,el.chatUserId,el.projectId,el.outputData,b.botTypeId 
FROM EngineLog el 
RIGHT OUTER JOIN Bot b 
ON el.botId = b.id
where b.botTypeId is not null
AND el.botId NOT IN (0) AND el.type = 'INFO'
AND NOT REGEXP_LIKE(inputData ,에임메드')  
group by el.createdAt
order by el.createdAt,el.botId,el.projectId,el.chatUserId
'''
cursor.execute(query)
conn.commit()

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

df = pd.read_sql(query, conn)

df = df.sort_values(['botId','chatUserId','projectId','botTypeId'])
df = df.reset_index()

/tmp/ipykernel_1679457/3807537479.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [4]:
# df[(df['botId']==390)&(df['projectId']==3)]

In [5]:
class User_daily:
    def __init__(self,start_At_list=[],end_At_list=[],dialogue_list=[],dialogue=[],dialog_count=[]):
        self.start_At_list = start_At_list
        self.end_At_list = end_At_list
        self.dialogue_list = dialogue_list
        self.dialog_count = dialog_count
        
    def list_append(self,start,end,dialogue):
        self.start_At_list.append(start)
        self.end_At_list.append(end)
        self.dialogue_list.append(dialogue)
        return self.start_At_list,self.end_At_list,self.dialogue_list
        
    def daily_date(self,key):
        self.start_answer = df['outputData'][key].count("\"answer\"")
        self.end_answer = df['outputData'][key+1].count("\"answer\"")

def Using_static_daily():
    Us_list = User_daily() #-1값 처리필요
    for idx in range(len(df)-1):
        Us_list.daily_date(idx)
        if(idx == len(df)-1):
            Us_list.list_append('-','-',0)
            continue
        else:
            try:
                if(Us_list.start_answer==1 and Us_list.end_answer==1):
                    Us_list.list_append(df.at[idx,'createdAt'],df.at[idx,'createdAt'],1)
                elif(Us_list.start_answer==1 and Us_list.end_answer!=1):
                    Us_list.list_append(df.at[idx,'createdAt'],df.at[idx+1,'createdAt'],Us_list.start_answer)
                elif(Us_list.start_answer!=1 and Us_list.end_answer==1):
                    Us_list.list_append("-",df.at[idx,'createdAt'],Us_list.start_answer)
                elif(Us_list.start_answer!=1 and Us_list.end_answer!=1):
                    Us_list.list_append("-","-",Us_list.start_answer)
                else:
                    Us_list.list_append(df.at[idx,'createdAt'],df.at[idx,'createdAt'],Us_list.start_answer)
            except IndexError as e:
                print("error")
        for dialog in Us_list.dialogue_list:
            if(dialog==1):
                Us_list.dialog_count.append(1)
            else:
                Us_list.dialog_count.append('-')
    df_st = pd.DataFrame(list(zip(df['botId'],df['chatUserId'],df['projectId'],Us_list.start_At_list,Us_list.end_At_list,Us_list.dialog_count,Us_list.dialogue_list,df['botTypeId'])), columns =['botId','chatUserId','projectId','startAt', 'endAt','dialog','dialogueCount','botTypeId']) 
    return df_st

In [6]:
df_st=Using_static_daily()
df_st.drop(df_st[(df_st['startAt'] =='-') & (df_st['endAt'] =='-')].index,inplace=True)
if(df_st['endAt'].iloc[-1]=='-'):
    df_st['endAt'].iloc[-1] = df_st['startAt'].iloc[-1]
else:
    df_st['startAt'].iloc[-1] = df_st['endAt'].iloc[-1]
df_st.reset_index(drop=True,inplace=True)

/tmp/ipykernel_1679457/1179838099.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_st['startAt'].iloc[-1] = df_st['endAt'].iloc[-1]


In [7]:
# df_st.drop(df_st[(df_st['startAt'] =='-') & (df_st['endAt'] =='-')].index,inplace=True)
# if(df_st['endAt'].iloc[-1]!='-'):
#     df_st['endAt'].iloc[-1] = df_st['startAt'].iloc[-1]
# else:
#     df_st['startAt'].iloc[-1] = df_st['endAt'].iloc[-1]
# df_st.reset_index(drop=True,inplace=True)

In [8]:
for i in range(len(df_st)):
    if(df_st.at[i,'dialogueCount']==1):
        df_st.at[i,'dialog']=1

In [9]:
for idx in range(0,len(df_st['endAt'])-1):
    if(df_st.at[idx+1,'startAt']=="-"):
        df_st.at[idx,'endAt']=df_st.at[idx+1,'endAt']
        df_st.at[idx,'dialogueCount']=df_st.at[idx+1,'dialogueCount']
    else:
        pass
df_st.drop(df_st[(df_st['startAt'] =='-')].index,inplace=True)

In [10]:
# for idx in range(0,len(df_st['endAt'])-1):
#     if(df_st.at[idx,'endAt']==("-")):
#         if(df_st['botId'][idx]==df_st['botId'][idx+1] and df_st['chatUserId'][idx]==df_st['chatUserId'][idx+1] and df_st['projectId'][idx]==df_st['projectId'][idx+1]):
#             df_st.at[idx,'endAt'] = df_st.at[idx+1,'endAt']
#             df_st.at[idx,'dialogueCount'] = df_st.at[idx+1,'dialogueCount']
#         else:
#             df_st.at[idx,'endAt'] = df_st.at[idx,'startAt']
#     else:
#         pass
# df_st.drop(df_st[(df_st['startAt'] =='-')].index,inplace=True)

In [11]:
## ToDate -- 구분일자
df_st['ToDate'] =  pd.to_datetime(df_st['startAt']).dt.date

In [12]:
# df_st.sort_values('ToDate')

In [13]:
## 일별 활성 유저수 DayUserCnt
## botId,projectId,chatUserId,botTypeId로 구분 된 값에서 botId,projectId,botTypeId로 집계 된 dialog의 개수
df_day = df_st.groupby(['ToDate','botId','projectId','chatUserId','botTypeId'],as_index=False)['dialog'].count().groupby(['ToDate','botId','projectId','botTypeId'],as_index=False)['dialog'].count()
df_day=df_day.rename(columns={"dialog": "DayUsrCnt"})

## 일별 세션 수 DaySesCnt
df_day['DaySesCnt']=df_st.groupby(['ToDate','botId','projectId','botTypeId'],as_index=False)['dialog'].count()['dialog']

##일별 채팅 수 DayChtCnt
df_day['DayChtCnt']=df_st.groupby(by=["ToDate","botId","projectId",'botTypeId'],as_index=False)['dialogueCount'].sum()['dialogueCount']


In [14]:
df_day['AccChtCnt']=df_day.groupby(by=['projectId','botTypeId'],as_index=False)['DayChtCnt'].cumsum()
df_day['AccSesCnt']=df_day.groupby(by=['projectId','botTypeId'],as_index=False)['DaySesCnt'].cumsum()

In [15]:
df_st.reset_index(drop=True,inplace=True)
df_st.head()

,botId,chatUserId,projectId,startAt,endAt,dialog,dialogueCount,botTypeId,ToDate
0,348,3133,1,2023-07-07 15:19:19.108000,2023-07-07 15:21:06.535000,1,6,2,2023-07-07
1,348,3133,1,2023-07-14 14:39:49.302000,2023-07-14 14:39:55.801000,1,3,2,2023-07-14
2,348,3415,1,2023-07-12 21:08:55.726000,2023-07-12 21:09:12.058000,1,2,2,2023-07-12
3,348,3460,1,2023-07-19 15:51:06.854000,2023-07-19 15:51:16.148000,1,2,2,2023-07-19
4,348,3466,1,2023-10-30 09:16:41.834000,2023-10-30 09:16:51.739000,1,3,2,2023-10-30


In [16]:
df_st['startAt'][0].round(freq='S')-df_st['endAt'][0].round(freq='S')

Timedelta('-1 days +23:58:12')

In [17]:
from datetime import datetime, timedelta
from decimal import Decimal

#일별 세션 타임 SesTime
# df_st['endAt']=df_st['endAt'].apply(lambda x:x.round(freq='S'))
# df_st['startAt']=df_st['startAt'].apply(lambda x:x.round(freq='S'))
df_st['SesTime']=df_st['endAt']-df_st['startAt']
df_test=df_st.copy()

In [18]:
# df_st['SesTime']=df_st['SesTime'].apply(lambda x:x.round(freq='S'))

In [19]:
#SesTime Decimal로 저장
for sestime in range(len(df_st)):
   df_st.at[sestime,'SesTime'] = df_st.at[sestime,'SesTime'].total_seconds()
   df_st.at[sestime,'SesTime'] =Decimal(str(df_st.at[sestime,'SesTime']))
# for sestime in range(len(df_st)):
#     if (str(df_st.at[sestime,'SesTime'])=='0 days 00:00:00'):
#         df_st.at[sestime,'SesTime']= None
#     else:
#         pass
        
# df_day['SesTime']=df_st.groupby(['ToDate','projectId','botTypeId'],as_index=False)['SesTime'].mean()['SesTime']

# df_test=df_day.copy()

In [20]:
df_test=df_st.copy()

In [21]:
df_test['SesTime']=df_test['SesTime'].astype(int)

In [22]:
for sestime in range(len(df_st)):
    if (str(df_st.at[sestime,'SesTime'])=='0.0'):
        df_st.at[sestime,'SesTime']= None
    if (df_test.at[sestime, 'SesTime'] > 1800):
        df_st.at[sestime, 'SesTime'] = 1800
    else:
        pass

In [23]:
df_stt=df_st.sort_values('ToDate')

In [24]:
df_day['SesTime']=df_st.groupby(['ToDate','botId','projectId','botTypeId'],as_index=False)['SesTime'].sum()['SesTime']

In [25]:
df_day['botCount']=df_st.groupby(['ToDate','botId','projectId','botTypeId'],as_index=False)['SesTime'].count()['SesTime']

In [26]:
df_st['dup']=df_st.duplicated(subset=['botId','projectId','chatUserId'])

In [27]:
for idx in range(len(df_st)):
    if(df_st.at[idx,'dup']==False):
        df_st.at[idx,'dup']=1
    else:
        df_st.at[idx,'dup']=0

/tmp/ipykernel_1679457/3815981404.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_st.at[idx,'dup']=1


In [28]:
##일별 신규 유저 수 NewUsrCnt
df_day['NewUsrCnt']=df_st.groupby(by=['ToDate','botId','projectId','botTypeId'],as_index=False)['dup'].sum()['dup']

##누적 신규 유저 수 AccNewUsrCnt
df_day['AccNewUsrCnt']=df_day['NewUsrCnt'].cumsum()

In [29]:
df_day = df_day.fillna(0.0)

In [31]:
for i in range(len(df_day)):
    query = "INSERT INTO StaticDaily(ToDate,botId,projectId,botTypeId,botCount,createdId,updatedId,DayChtCnt,AccChtCnt,DayUsrCnt,DaysesCnt,AccSesCnt,NewUsrCnt,AccNewUsrCnt,SesTime) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    values = df_day['ToDate'][i], df_day['botId'][i], df_day['projectId'][i], df_day['botTypeId'][i],df_day['botCount'][i],1,1,df_day['DayChtCnt'][i],df_day['AccChtCnt'][i], df_day['DayUsrCnt'][i], df_day['DaySesCnt'][i], df_day['AccSesCnt'][i], df_day['NewUsrCnt'][i], df_day['AccNewUsrCnt'][i], df_day['SesTime'][i]
    cursor.execute(query,values)
    conn.commit()

In [32]:
'''
week
'''

'\nweek\n'

In [33]:
df_st = df_day
df_st['week'] = df_day['ToDate']
df_test['week'] = df_day['ToDate']
for week in range(len(df_st)):
    df_st.at[week,'week']=df_day['ToDate'][week].isocalendar()[1]  
    df_test.at[week,'week']=df_day['ToDate'][week].isocalendar()[1]  

df_st['day']=df_st['ToDate']
df_test['day']=df_st['ToDate']

In [34]:
df_week = df_st.groupby(by=['ToDate','day','week','projectId','botId','botTypeId'],as_index=False).first()
df_week=df_week.drop(columns=['DayChtCnt','DayUsrCnt','AccChtCnt','DaySesCnt','AccSesCnt','AccNewUsrCnt','NewUsrCnt'])

In [35]:
## WeekChtCnt 주별 대화 수 
df_week['WeekChtCnt'] = df_st.groupby(by=['day','week','projectId','botId','botTypeId'],as_index=False)['DayChtCnt'].sum()['DayChtCnt']
## WeekUsrCnt 주별 활성 이용자
df_week['WeekUsrCnt']=df_st.groupby(by=['day','week','projectId','botId','botTypeId'],as_index=False)['DayUsrCnt'].sum()['DayUsrCnt']
## WeekSesCnt 주별 세션 수 
df_week['WeekSesCnt']=df_st.groupby(by=['day','week','projectId','botId','botTypeId'],as_index=False)['DaySesCnt'].sum()['DaySesCnt']
## WeekNewUsr 주별 신규 유저 수 
df_week['WeekNewUsr']=df_st.groupby(by=['day','week','projectId','botId','botTypeId'],as_index=False)['NewUsrCnt'].sum()['NewUsrCnt']
# df_week=df_week.sort_values('day')
df_week=df_week.reset_index(drop=True)

In [36]:
#EndDate
from pandas.tseries.offsets import *
df_week['ToDate'][0]+Week(weekday=0)
# df_week['week'] = pd.to_datetime(df_week['week'])

for date in range(len(df_week)):
    if(df_week.at[date,'week']!=20):
        df_week.at[date,'EndDate']=df_week.at[date,'ToDate']+Week(weekday=0)+timedelta(days=-1)
    else:
        df_week.at[date,'EndDate']=df_week.at[date,'ToDate']+Week(weekday=6)
    # else:
    #     df_week.at[date,'EndDate']=df_week.at[date,'ToDate']+Week(weekday=3)
# StartDate
df_week.rename(columns = {'ToDate' : 'StartDate'}, inplace = True)

In [37]:
# for sestime in range(len(df_day)):
#     # df_day.at[sestime,'SesTime'] = df_day.at[sestime,'SesTime'.total_seconds()]
#    df_week.at[sestime,'SesTime'] = df_week.at[sestime,'SesTime'].total_seconds()
#    df_week.at[sestime,'SesTime'] =Decimal(str(df_week.at[sestime,'SesTime']))
#2205

In [38]:
df_week['SesTime']=df_week.groupby(['week','botId','projectId','botTypeId'],as_index=False)['SesTime'].sum()['SesTime']


In [39]:
## 주별 누적 채팅 수 
df_week['WeekAccChtCnt']=df_week.groupby(by=['projectId','botTypeId'],as_index=False)['WeekChtCnt'].cumsum()
## 주별 누적 활성 유저 수 
df_week['WeekAccUsrCnt']=df_week['WeekUsrCnt'].cumsum()
## 주별 누적 신규 유저 수 
df_week['WeekAccNewUsr']=df_week['WeekNewUsr'].cumsum()
## 주별 누적 세션 수 
df_week['WeekAccSesCnt']=df_week.groupby(by=['projectId','botTypeId'],as_index=False)['WeekSesCnt'].cumsum()

In [40]:
len(df_day)

1492

In [41]:
df_week = df_week[['week','StartDate','EndDate','botId','projectId','botTypeId','botCount','WeekChtCnt','WeekAccChtCnt','WeekUsrCnt','WeekAccUsrCnt','WeekSesCnt','WeekAccSesCnt','WeekNewUsr','WeekAccNewUsr','SesTime']]

In [42]:
df_week=df_week.fillna(0.0)

In [43]:
for i in range(len(df_week)):
    query = "INSERT INTO StaticWeekly(StartDate,EndDate,week,createdId,updatedId,botId,projectId,botTypeId,botCount,WeekChtCnt,WeekAccChtCnt,WeekUsrCnt,WeekAccUsrCnt,WeekSesCnt,WeekAccSesCnt,WeekNewUsr,WeekAccNewUsr,SesTime) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    values = df_week['StartDate'][i], df_week['EndDate'][i],df_week['week'][i],1,1,df_week['botId'][i],df_week['projectId'][i], df_week['botTypeId'][i],df_week['botCount'][i],df_week['WeekChtCnt'][i], df_week['WeekAccChtCnt'][i], df_week['WeekUsrCnt'][i],df_week['WeekAccUsrCnt'][i],df_week['WeekSesCnt'][i], df_week['WeekAccSesCnt'][i], df_week['WeekNewUsr'][i], df_week['WeekAccNewUsr'][i],df_week['SesTime'][i]
    cursor.execute(query,values)
    conn.commit()

In [44]:
# df_week.to_sql('StaticWeekly', con=conn, if_exists='replace', index=False)
#1888

In [45]:
df_day.loc[3:19][(df_day['projectId']==3)&(df_day['botTypeId']==2)]['botCount'].sum()

/tmp/ipykernel_1679457/4236121658.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_day.loc[3:19][(df_day['projectId']==3)&(df_day['botTypeId']==2)]['botCount'].sum()


22